In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import io

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import geopandas as gpd
from shapely.geometry import Polygon


# import k-means from clustering stage
from sklearn.cluster import KMeans

# for normalization 
from sklearn import preprocessing

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [389]:
conda install -c anaconda wget 

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - wget


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.8.28  |                0         133 KB  anaconda
    certifi-2019.9.11          |           py37_0         154 KB  anaconda
    conda-4.7.12               |           py37_0         3.0 MB  anaconda
    openssl-1.1.1d             |       h1de35cc_2         3.4 MB  anaconda
    wget-1.20.1                |       h051b688_0         478 KB  anaconda
    ------------------------------------------------------------
                                           Total:         7.2 MB

The following NEW packages will be INSTALLED:

  wget               anaconda/osx-64::wget-1.20.1-h051b688_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                          

In [3]:
address = 'Toronto, Ontario' ## the city for the business

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


## Business Improvement Areas (BIA)
https://open.toronto.ca/dataset/business-improvement-areas/

In [4]:
#BIA= pd.read_csv('Business Improvement Areas Data.csv')
BIA = gpd.read_file('Business Improvement Areas Data.geojson.json')
BIA.head()

,_id,AREA_ID,DATE_EFFECTIVE,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,1067,2478937,2019-05-28T21:47:59,26004921,None,020-01,020-01,Little Italy,Little Italy,311705.037,4835053.901,-79.414394,43.655397,17536177,2.323416e+05,3917.542802,(POLYGON ((-79.42050104104892 43.6551985070524...
1,1068,2478936,2019-05-28T21:47:59,26004920,None,042-01,042-01,Liberty Village,Liberty Village,311152.727,4833083.985,-79.421265,43.637670,17536193,7.972921e+05,4400.913504,(POLYGON ((-79.42465920454276 43.6393767780487...
2,1069,2478935,2019-05-28T21:47:59,26004919,None,093-01,093-01,Leslieville,Leslieville,318224.026,4835848.463,-79.333555,43.662460,17536209,3.513029e+05,6457.749078,(POLYGON ((-79.3240995497234 43.66505000369194...
3,1070,2478934,2019-05-28T21:47:59,26004918,None,033-00,033-00,Kennedy Road,Kennedy Road,322480.719,4847294.113,-79.280401,43.765395,17536225,1.830952e+06,13312.406690,(POLYGON ((-79.28586522957964 43.7770659504300...
4,1071,2478933,2019-05-28T21:47:59,26004917,None,002-00,002-00,Lakeshore Village,Lakeshore Village,304525.971,4829026.074,-79.503394,43.601170,17536241,1.719640e+05,3336.398221,(POLYGON ((-79.50761897869687 43.6002212031379...


In [5]:
map_T = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.GeoJson(BIA.geometry).add_to(map_T)

popup = folium.Popup(BIA.AREA_NAME).add_to(map_T)
map_T

In [6]:

map_T.save('BIA.html')

## Subway stations data
### https://www.arcgis.com/home/item.html?id=baa25877f51443bdad50fc39f1cc57a1

In [7]:
subway=pd.read_csv('TorontoSubwayStations_Ridership.csv')
subway.rename(columns={"y": "latitude", "x": "longitude",
                       "Station Name": "station_name",
                       "Total Daily Riders":"daily_riders"}, errors="raise", inplace=True)
subway.head()

,OBJECTID_1,Wheelchair Access?,station_name,To Trains,From Trains,daily_riders,Opened,longitude,latitude
0,1,Yes,Finch,48380,49078,97458,3/29/1974 4:00:00 AM,-79.415291,43.780847
1,2,Yes,North York Centre,12037,12152,24189,6/18/1987 4:00:00 AM,-79.412755,43.767867
2,3,Yes,Sheppard-Yonge (line 4),23634,24635,48269,11/22/2002 5:00:00 AM,-79.411691,43.761647
3,4,Yes,York Mills,12609,12886,25495,3/30/1973 5:00:00 AM,-79.406490,43.744847
4,5,No,Lawrence,12443,11423,23866,3/30/1973 5:00:00 AM,-79.402690,43.725948


##### analysis for the subway stations data

In [208]:
subway.describe()

,OBJECTID_1,To Trains,From Trains,daily_riders,longitude,latitude
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,37.500000,17647.540541,18562.500000,36210.040541,-79.388878,43.693718
std,21.505813,19080.897119,19787.948184,38828.704566,0.063270,0.042763
min,1.000000,624.000000,566.000000,1190.000000,-79.535993,43.637148
25%,19.250000,6903.750000,6841.250000,13887.750000,-79.417769,43.660347
50%,37.500000,12240.000000,13854.000000,25837.000000,-79.391153,43.677594
75%,55.750000,18892.750000,20658.000000,38715.000000,-79.368089,43.725610
max,74.000000,101588.000000,109691.000000,211279.000000,-79.251654,43.780847


In [8]:
#normalize the number of daily riders to reflect it on the map as the radius of the circle marker
# Create x, where x the 'scores' column's values as floats
x = subway[['daily_riders']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
subway['scaled_riders']=x_scaled

In [9]:
# add the stations to the map
# add markers to map
map_T = folium.Map(location=[latitude, longitude], zoom_start=11)
#folium.GeoJson(BIA.geometry).add_to(map_T)
for lat, lng, name,num in zip(subway['latitude'], subway['longitude'], subway['station_name'],subway['scaled_riders']):
        label = folium.Popup(name, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=num,
        popup=label,
        color='purple',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_T)  
    
map_T

In [10]:
map_T.save('Subway.html')

## Parking lot faicilities
https://open.toronto.ca/dataset/parking-lot-facilities/

In [14]:
parking=pd.read_excel('parking-lot-facilities-q3-2016.xlsx')
parking.head()

,Parking Lot Asset ID,Park Name,Parking Spaces,Handicap Parking Spaces,GIS Coordinate,Access
0,32202,ADAMS PARK,134.0,6.0,"(-79.1447873015202,43.7924791844875)",Public
1,32203,AGINCOURT PARK,300.0,9.0,"(-79.2751862118817,43.7877827155571)",Public
2,32524,ALBION ARENA - Building Grounds,118.0,10.0,"(-79.5818486224219,43.7394116555141)",Public
3,32204,ALLAN A. LAMPORT STADIUM PARK,360.0,3.0,"(-79.4231327178795,43.6381423188339)",Public
4,32205,ALLAN GARDENS,6.0,2.0,"(-79.3748943065984,43.6615560937226)",Public


In [15]:
#dropping any non public parking
parking.drop(parking[ parking['Access'] != 'Public' ].index , inplace=True)
parking.reset_index(drop=True, inplace=True)
#lat and long 

parking['longitude'] = ""
parking['latitude'] = ""

for  i in range(0,len(parking),1):
    parking['longitude'][i]=parking['GIS Coordinate'][i].strip('()').split(',')[0]
    parking['latitude'][i]=parking['GIS Coordinate'][i].strip('()').split(',')[1]

    
parking['latitude'] = parking['latitude'].astype(float)
parking['longitude'] = parking['longitude'].astype(float)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [21]:
# add parking markers to map
map_T = folium.Map(location=[latitude, longitude], zoom_start=12)
#folium.GeoJson(BIA.geometry).add_to(map_T)

for lat, lng, name in zip(parking['latitude'], parking['longitude'], parking['Park Name']):
        label = folium.Popup(name, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='darkblue',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_T)  


    
map_T

In [22]:
map_T.save('Parking.html')

In [20]:

# add parking markers to map
map_T = folium.Map(location=[latitude, longitude], zoom_start=12)
#folium.GeoJson(BIA.geometry).add_to(map_T)

#for lat, lng, name in zip(parking['latitude'], parking['longitude'], parking['Park Name']):folium.Marker([lat,lng],popup=name,icon=folium.Icon(color='red', icon='info-sign')).add_to(map_T)  
#for  i in range(0,len(parking),1):folium.Marker(location=[parking['latitude'][i],parking['longitude'][i]],popup=parking['Park Name'][i],icon=folium.Icon(color='red', icon='info-sign')).add_to(map_T)  


    
for lat, lng, name,num in zip(subway['latitude'], subway['longitude'], subway['station_name'],subway['scaled_riders']):
        label = folium.Popup(name, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=num,
        popup=label,
        color='purple',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_T)  
        


    
map_T

## zillow

In [134]:
conda install -c auto pyzillow

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - pyzillow

Current channels:

  - https://conda.anaconda.org/auto/osx-64
  - https://conda.anaconda.org/auto/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [ ]:
https://www.realtor.ca/map#ZoomLevel=10&Center=43.708087%2C-79.376385&LatitudeMax=43.98738&LongitudeMax=-78.63343&LatitudeMin=43.42749&LongitudeMin=-80.11934&CurrentPage=2&Sort=1-A&GeoIds=g30_dpz89rm7&GeoName=Toronto%2C%20ON&PropertyTypeGroupID=2&PropertySearchTypeId=0&TransactionTypeId=1